In [ ]:
# Mount your Google Drive to this notebook
# The purpose is to allow your code to access to your files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change the directory to your own working directory
# Any files under your working directory are available to your code
# TO DO: enter the name of your directory
import os
os.chdir('/content/drive/MyDrive/5554/GoogleMapAPI')

In [ ]:
pip install google_streetview

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for google-streetview: filename=google_streetview-1.2.9-py3-none-any.whl size=9797 sha256=4c5fb636b9d01f1aee2ed1dacdee6af7ec0eb23bbef857354a4efbbc00962744
  Stored in directory: /root/.cache/pip/wheels/3d/3c/22/66891d135d7de956a903e1eac58515b816e499df0a6f486c28
  Created wheel for kwconfig: filename=kwconfig-1.1.7-py3-none-any.whl size=4990 sha256=aa61f042bf98676134529b11bdcc704173379d1e36c8123ada0ea707a9fafdf1
  Stored in directory: /root/.cache/pip/wheels/75/99/be/6216a0ec525d7bf335628c0ac2463f9a24357c0f07870db419
Successfully built google-streetview kwconfig


In [ ]:
import google_streetview.api
import pandas as pd
import numpy as np
import random# Use 'conda install shapely' to import the shapely library.
from shapely.geometry import Polygon, Point
from google_streetview import helpers
from os import path, makedirs
from pprint import pprint
try:
  from urllib.parse import urlencode
except ImportError:
  from urllib import urlencode

import json
import requests

In [ ]:
#Lat & Long points that encompass Singapore
poly = Polygon([(1.228146735701861, 103.60746204587528),
                (1.3474212549956137, 103.63750279150415),
                (1.426121084056379, 103.67603228628388),
                (1.4492327705114603, 103.72601417713122),
                (1.4467064034530708, 103.7705673236209),
                (1.469630743816576, 103.80800694228734),
                (1.4302323626673925, 103.87793329552278),
                (1.420918840813145, 103.91204276098085),
                (1.3907832734384507, 103.99134048973873),
                (1.3588634445172736, 104.03322587221558),
                (1.3097815977301719, 104.01245484238075),
                (1.318019178589225, 103.9709127827111),
                (1.2694513861811194, 103.86053449927056),
                (1.2423354003442755, 103.83272535188841),
                (1.2811214696711377, 103.75341778221673),
                (1.2301502764015833, 103.67634181112543)])

In [ ]:
#Given the polygon created by the lat/long points. Chooses a randomly selected point inside to select a streetview image from
# Link to source: https://medium.com/the-data-journal/a-quick-trick-to-create-random-lat-long-coordinates-in-python-within-a-defined-polygon-e8997f05123a
def polygon_random_points (poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    while len(points) < num_points:
        random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            points.append(random_point)
            return points# Choose the number of points desired. This example uses 20 points. 



In [ ]:
#Generate Parameters for google streetview api based on randomly selected lat/long
def gen_api_parameters():
  points = polygon_random_points(poly,1)# Printing the results.
  for p in points:
      lat = p.x
      lon = p.y
  params = [{
    'size': '640x640', # max 640x640 pixels
    'location': '%s,%s'%(round(lat,6),round(lon,6)),#'46.414382,10.013988',
    'heading': '45',
    'pitch': '-0.76',
    'radius': '1000',
    'key': '',
    'source': 'outdoor'
  }]
  return params


In [ ]:
def download_links2(self, dir_path, metadata_file='metadata.json', metadata_status='status', status_ok='OK'):
    """Download Google Street View images from parameter queries if they are available.
    
    Args:
      dir_path (str):
        Path of directory to save downloads of images from :class:`api.results`.links
      metadata_file (str):
         Name of the file with extension to save the :class:`api.results`.metadata
      metadata_status (str):
        Key name of the status value from :class:`api.results`.metadata response from the metadata API request.
      status_ok (str):
        Value from the metadata API response status indicating that an image is available.
    """
    metadata = self.metadata
    if not path.isdir(dir_path):
      makedirs(dir_path)
    
    # (download) Download images if status from metadata is ok
    for i, url in enumerate(self.links):
      if metadata[i][metadata_status] == status_ok:
        file_name = random.randrange(0,100000)
        file_path = path.join(dir_path, 'gsv_' + '%s'%(file_name) + '.jpg')
        metadata[i]['_file'] = path.basename(file_path) # add file reference
        helpers.download(url, file_path)
    
    # (metadata) Save metadata with file reference
    metadata_path = path.join(dir_path, metadata_file)
    with open(metadata_path, 'w') as out_file:
      json.dump(metadata, out_file)

In [ ]:
for i in range(0,5000):
  params = gen_api_parameters()
  results = google_streetview.api.results(params)
  download_links2(results,'singapore')


KeyboardInterrupt: ignored

In [ ]:
results.preview()

In [ ]:
#download_links2(results,'downloads', metadata_file='metadata.json')